## KenLm

统计语言模型工具有比较多的选择，目前使用比较好的有srilm及kenlm，其中kenlm比srilm晚出来，训练速度也更快，而且支持单机大数据的训练。现在介绍一下kenlm的使用方法。语言行业最大的改进是增加了新的语言模型KenLM，它快速，节省内存，最重要的是，允许在开源许可下使用多核处理器。

在我们项目中使用KenLM主要是来训练n-gram的语言模型。

### Kenlm在Windows上的安装和使用

 Kenlm在Windows上安装和使用是一个大坑，这个说起来就是一把心酸鼻涕泪了，总之会遇到非常多的坑，这里记录下我在使用过程。kenLM 在windows上需要使用Cygwin64模拟linux安装，所以
 
**1. 首先安装Cygwin64**

傻瓜式安装后选择要下载的软件和版本,镜像用163即可，很快。尽量选择wget方便下载kenlm。然后安装比较重要的包 :cmake make gcc gcc-g++ gdb  gcc-core  boost boost-devel zlib zlib-deved 。Cygwin64安装中有很多问题百度就可以解决，不再赘述。


**2. 编译kenlm**
去git上clone或者直接下载release的包，解压到cygwin64\目录下。
其中可能缺少各类依赖包和环境，一步一步百度解决。

```
mkdir kenlm/build
cd kenlm/build && cmake .. && make -j8
cd kenlm/build && make install 
```

**3. IDE中使用kenlm加载模型**
此时进入kenlm/目录下，可以查看到有一个setup.py，在此执行python install ./setup.py后，将一下目录和文件放置到项目依赖的第三方库中

```
kenlm-0.0.0.dist-info\
kenlm-0.0.0-py3.7-win-amd64.egg\
pypi_kenlm-0.1.20190403-py3.7-win-amd64.egg\
kenlm.cp37-win_amd64.pyd
```

比如我是将这些文件复制粘贴到我的Anaconda中叫做SpellCheck的lib文件夹中：*E:\AnaConda\envs\SpellCheck\Lib\site-packages*。
最后在项目中执行kenlm中example.py的案例，查看是否执行成功，未能正常执行成功，继续回溯问题。

### 自定义语言模型

首先了解什么是语言模型。很多数据挖掘的读者都已经听说过Word2Vec，知道它是一个能够生成词向量的工具，很多人也知道将词向量作为模型的特征来进行输入。但相信不少读者不知道为什么会有词向量，为什么Word2Vec能生成词向量。Word2Vec本身的光芒（Google出品、速度快、效果也不错、在Python中有很好实现等）已经把同类产品以及背后的原理都给掩盖下去了。事实上，词向量的初衷，是为了更好地生成语言模型，最经典的文章应该是深度学习的鼻祖之一——Bengio——的《A Neural Probabilistic Language Model》。这一段的重点是谈语言模型，不是词向量。关于词向量，有兴趣的读者可以参考下面的文章：

[Deep Learning in NLP （一）词向量和语言模型](http://licstar.net/archives/328)

[火光摇曳的《我们是这样理解语言的》系列](http://www.flickering.cn/?s=我们是这样理解语言的)

语言模型是计算条件概率:

$$p(w_n|w_1, w_2, ..., w_{n-1})$$

的模型，其中$w_1, w_2, ..., w_{n-1}$使句子中的前n-1个词。语言模型在很多方面都有应用，比如说分词、语音识别、机器翻译等。为了得到语言模型，有很多方法，比如说最简单的是“统计+平滑”的方法，还有最大熵语言模型、CRF语言模型等，而当前深度学习框架下研究得很多的是“神经网络语言模型”，它的大概思路是：

$p(w_n|w_1, w_2, ..., w_{n-1})$是关于$w1,w2,…,wn$的一个函数，这个函数的具体形式我不知道，所以利用神经网络去拟合它，为了更好地拟合，并且减少模型参数，还把词语“嵌入”到实数空间中，用短向量来表示词语，跟语言模型一起训练。从这个角度看，词向量只是语言模型的副产品。语言模型生成的词向量能够较好地表示语义，对人类来说，语义是一种推理和理解的过程，而我们的语言模型，就是从前n−1个字推测下一个字，这也是一个推理的过程。既然包含了推理成分在里边，就有可能捕捉到语义了。

### 无监督分词

如果一个长度为l的字符串,用贝叶斯公式对词进行切分，事实上都对应着l个条件概率的相乘，我们就是从这些条件概率的相乘模式中，找出结果最大的那个。而同样的，如果我们知道了最优的相乘模式，就可以对应地写出分词结果来。一般使用4-gram语言模型进行分词效果已经很不错了。

更系统地看，其实就是将分词转化为了标注问题，如果字语言模型取到4-gram，那么它相当于做了如下的字标注：

```
b：单字词或者多字词的首字
c：多字词的第二字
d：多字词的第三字
e：多字词的其余部分

```

对于句子中的一个字$s_k$说，就有

$$ p(b)=p(s_k)$$

$$ p(c)=p(s_k|s_{k−1})$$

$$ p(d)=p(s_k|s_{k−2}s{k−1})$$

$$ p(e)=p(s_k|s_{k−3}s_{k−2}s_{k−1}) $$

这就是将分词问题变成了一种字标注问题，而每个标签的概率由语言模型给出。而且，显然b后面只能接b或者c，类似地，就得到非零的转移概率只有：

$$p(b|b),p(c|b),p(b|c),p(d|c),p(b|d),p(e|d),p(b|e),p(e|e)$$

这些转移概率的值，决定了划分出来的是长词还是短词。最后找最优路径，依旧由viterbi算法完成。

到这里，问题就变成了语言模型的训练了，这是无监督的。我们只需要花心思优化语言模型，而这方面不论是理论还是实战都已经很成熟了，有不少现成的工具可以用。简单地可以只用传统的“统计+平滑”模型，如果要从语义来做，那么就可以用最新的神经语言模型。总而言之，分词的效果，取决于语言模型的质量。

### 实现

之前使用了HMM_Model.py实现了一个使用简单分词的拼音转汉字的算法。这次结合KenLm省内存、算法快的特点加载一个自定义的语言模型，由viterbi算法找出最优解，将拼音转为汉字。

一般使用KenLM构建中文语言模型流程有：

* 数据处理
* 原始语料的构建（通过脚本的方式）
* 数据的预处理与数据清洗
* 非汉字字符将被删除，阿拉伯数字转换为汉字，比如“1000”转换为“一千”；
* 重复的空白字符被压缩为一个，并且开始的空白字符将被删除。
* 选择前xxx个最常用的单词来建立词表，其余部分替换为“UNKNOWNWORD”。
* 删除开始和结尾的空白字符。
* 删除英文标点和中文标点。
* 在两个字符之间插入空白字符。
* [可选]繁简转换：语料库中可能会有繁体，所以需要统一转换为简体
    方法一：OpenCC
    方案二：hanziconv
* 分词：jieba分词
* 再次清洗：分词后可能还会有空白符，再次去除
* 保存处理后的文本语料库

然后进行模型训练：使用KenLM来训练n-gram语言模型[与DeepSpeech2兼容]
进入build/目录，执行操作：

    bin/lmplz -o 3 --verbose header --text mytext/19-01-au.txt --arpa mylmmodel/test.arpa
    -o 为必须的参数，3表示3-gram
    --verbose header表示向ARPA文件开头添加详细的头文件（可选）
    --text 训练的语料库
    --arpa 训练生成的语言模型，在这之后为了减少空间可以将其变成二进制格式的
    19-01-au.txt为从Full-text corpus data下载的英文训练样例
    将生成的语言模型变成二进制形式
    为减少空间，需要将生成的.arpa格式的语言模型变成二进制格式的.bin或.klm格式的：
    bin/build_binary trie -a 22 -q 8 -b 8 mylmmodel/log3.arpa mylmmodel/log3.bin
    bin/build_binary trie -a 22 -q 8 -b 8 mylmmodel/log3.arpa mylmmodel/log3.klm
    为了节省磁盘存储空间，我们将使用参数“-a 22 -q 8 -b 8”将 arpa 文件转换为“trie”二进制文件。“-a”表示在“trie”中用于切分的指针的最高位数。“-q -b”是概率和退避的量化参数。


我这里直接使用的千兆中文文本训练的中文语言模型[zh_giga.no_cna_cmn.prune01244.klm(2.8G)](https://deepspeech.bj.bcebos.com/zh_lm/zh_giga.no_cna_cmn.prune01244.klm)

In [6]:
import kenlm
import numpy as np
from utils.common import *

model_path = "D:\\PyWorkspace\\LanguageAndSpeechRecogintion\\data\\zh_giga.no_cna_cmn.prune01244.klm"
model = kenlm.Model(model_path)
min_prob = -1e+3

pinyin2word = get_symbol_list()

word2id = word2id()
id2word = dict(zip(word2id.values(), word2id.keys()))

pinyin2id = pinyin2id()
id2pinyin = dict(zip(pinyin2id.values(), pinyin2id.keys()))

def get_word_in_pi_prob_use_kenlm(word_id):
    '''
    返回某一个隐藏状态（汉字）在初始矩阵的概率，
    如果没有则为min_word_prob
    :param word_id:
    :return:
    e.g.
    word:还, acc:-6.3978071212768555
    '''
    word = id2word[word_id]
    if not word in model:
        return min_prob
    return model.score(word)


def get_hidden_to_observer_emit_prob_use_kenlm(pinyin, word):
    '''
    隐藏状态到观察状态（字+拼音）组合的发射矩阵
    :param pinyin:guo2
    :param word: 国
    :return:
    e.g.
    word:害, acc:-5.514978885650635, per:0.31204276487005345
    word:海, acc:-5.761005878448486, per:0.32596320675334983
    word:还, acc:-6.3978071212768555, per:0.36199402837659667
    '''
    word_list = pinyin2word[pinyin]
    single_word_score = model.score(word)
    total = sum([model.score(w) for w in word_list])
    return single_word_score/total


def get_trans_prob_use_kenlm(*args):
    '''
    获取转移概率, 使用log函数做处理，分数越接近0，转移概率越高
    :param phrase: 词组类似：中国/钟国/忠国
    :return: -9.256282567977905
    '''
    word_list = args
    state = kenlm.State()
    state1 = kenlm.State()
    model.NullContextWrite(state)

    acc = 0.0
    for index, word in enumerate(word_list):
        if index % 2 == 0:
            acc += model.BaseScore(state, word, state1)
        else:
            acc += model.BaseScore(state1, word, state)
    return acc


def viterbi_use_kenlm_model(word_list, pinyin_list, n):
    """
    维特比算法求解最大路径问题
    :param word_list:   每个拼音对应的隐藏状态矩阵
    :param n:   可能观察到的状态数， 对应为汉字数量
    :param id2word:    id到汉字的映射
    :return:
    """
    T = len(word_list)  # 观察状态的长度
    delta = np.zeros((T, n))  # 转移值
    psi = np.zeros((T, n), dtype=int)  # 转移下标值

    # 初始化第一个字符的隐藏初始状态概率， 设置为每个词在词典中的单独出现的概率
    words = word_list[0]
    for i, w in enumerate(words):
        delta[0][i] = abs(get_word_in_pi_prob_use_kenlm(w))

    # 动态规划计算
    for idx in range(1, T):
        words = word_list[idx]  # 第T时刻所有可能出现的字的集合
        for i in range(len(words)):
            # max_value = -1e+4
            max_value = 1e+4
            pre_words = word_list[idx-1]

            last_index = 0
            for j in range(len(pre_words)):
                tmp_key = id2word[pre_words[j]] + id2word[words[i]]  # 中国/钟国/忠国
                # 获得转移概率
                prob = abs(get_trans_prob_use_kenlm(id2word[pre_words[j]], id2word[words[i]]))

                # 前一时刻的字观察状态到隐藏状态的概率 * 转移概率
                tmp_value = delta[idx-1][j] * prob
                # if tmp_value > max_value:
                #     max_value = tmp_value
                #     last_index = j

                if tmp_value < max_value:
                    max_value = tmp_value
                    last_index = j

            # 计算观察状态到隐藏状态的概率
            emit_prob = abs(get_hidden_to_observer_emit_prob_use_kenlm(pinyin_list[idx], id2word[words[i]])) * max_value  # 观察状态到隐藏状态的概率 * 前一时刻所有字和当前字组合的最大概率

            delta[idx][i] = emit_prob
            psi[idx][i] = last_index  # 保存当前字的前一时刻 和当前字组合的最大概率 的下标值

    prob = 1e+4
    path = np.zeros(T, dtype=int)
    path[T-1] = 1

    # 获取最大的转移值
    desc_word_id = []
    fit_index = 0
    for i in range(n):
        if delta[T-1][i] != 0 and prob > delta[T-1][i]:
            prob = delta[T-1][i]
            path[T-1] = i
            fit_index = i
    desc_word_id.append(word_list[T-1][fit_index])

    # 最优路径回溯
    for t in range(T-2, -1, -1):
        last_index = psi[t+1][path[t+1]]
        path[t] = last_index
        desc_word_id.append(word_list[t][last_index])

    final_word = ""
    for id in reversed(desc_word_id):
        final_word += id2word[id]

    return final_word


def viterbi_use_kenlm_model_v1(word_list, pinyin_list, n):
    """
    维特比算法求解最大路径问题
    :param word_list:   每个拼音对应的隐藏状态矩阵
    :param n:   可能观察到的状态数， 对应为汉字数量
    :param id2word:    id到汉字的映射
    :return:
    """
    T = len(word_list)  # 观察状态的长度
    delta = np.zeros((T, n))  # 转移值
    psi = np.zeros((T, n), dtype=int)  # 转移下标值

    # 初始化第一个字符的隐藏初始状态概率， 设置为每个词在词典中的单独出现的概率
    words = word_list[0]
    for i, w in enumerate(words):
        delta[0][i] = abs(get_word_in_pi_prob_use_kenlm(w))

    # 动态规划计算
    for idx in range(1, T):
        words = word_list[idx]  # 第T时刻所有可能出现的字的集合
        for i in range(len(words)):
            max_value = 1e+4
            pre_words = word_list[idx-1]

            last_index = 0
            for j in range(len(pre_words)):
                if idx > 1:  # 3n-gram
                    pre_pre_word = word_list[idx-2]
                    for k in range(len(pre_pre_word)):
                        tmp_key = id2word[pre_pre_word[k]] + id2word[pre_words[j]] + id2word[words[i]]  # 中国地/种过地
                        # 获得转移概率
                        prob = abs(get_trans_prob_use_kenlm(id2word[pre_pre_word[k]], id2word[pre_words[j]], id2word[words[i]]))
                        # 前一时刻的字观察状态到隐藏状态的概率 * 转移概率
                        tmp_value = delta[idx - 1][j] * prob

                        if tmp_value < max_value:
                            max_value = tmp_value
                            last_index = j
                else:  # 2n-gram
                    tmp_key = id2word[pre_words[j]] + id2word[words[i]]  # 果地/过地/国地
                    # 获得转移概率
                    prob = abs(get_trans_prob_use_kenlm(id2word[pre_words[j]], id2word[words[i]]))
                    # 前一时刻的字观察状态到隐藏状态的概率 * 转移概率
                    tmp_value = delta[idx - 1][j] * prob

                    if tmp_value < max_value:
                        max_value = tmp_value
                        last_index = j

            # 计算观察状态到隐藏状态的概率
            emit_prob = abs(get_hidden_to_observer_emit_prob_use_kenlm(pinyin_list[idx], id2word[words[i]])) * max_value  # 观察状态到隐藏状态的概率 * 前一时刻所有字和当前字组合的最大概率

            delta[idx][i] = emit_prob
            psi[idx][i] = last_index  # 保存当前字的前一时刻 和当前字组合的最大概率 的下标值

    prob1 = 1e+4
    path = np.zeros(T, dtype=int)
    path[T-1] = 1

    # 获取最大的转移值
    desc_word_id = []
    fit_index = 0
    for i in range(n):
        if delta[T-1][i] != 0 and prob1 > delta[T-1][i]:
            prob1 = delta[T-1][i]
            path[T-1] = i
            fit_index = i
    desc_word_id.append(word_list[T-1][fit_index])

    # 最优路径回溯
    for t in range(T-2, -1, -1):
        last_index = psi[t+1][path[t+1]]
        path[t] = last_index
        desc_word_id.append(word_list[t][last_index])

    final_word = ""
    for id in reversed(desc_word_id):
        final_word += id2word[id]

    return final_word

In [8]:
tests = [
    ['zhong1', 'guo2', 'ren2', 'min2'],
    ['ming2', 'tian1', 'gao1', 'kao3'],
    ['hou4', 'tian1', 'jiu4', 'yao4', 'chu1', 'cheng2', 'ji4', 'le5'],
    ['xi1', 'wang4', 'ni3', 'hou4', 'tian1', 'gao1', 'kao3', 'shun4', 'li4'],
    ['wo3', 'ming2', 'tian1', 'yao4', 'chi1', 'ping2', 'guo3'],
    ['ni3', 'de5', 'bao4', 'jia4', 'tai4', 'gao1', 'le5'],
    ['qin2', 'lao2', 'yong3', 'gan3'],
    ['xin1', 'zhi1', 'du4', 'ming2'],
    ['jin1', 'tian1', 'zhuan4', 'le5', "hao3", 'duo1', 'qian2'],
    ['ni3', 'shi4', 'fou3', 'you3', "hao3", 'duo1', 'wen4', 'hao4']
]

for pinyin_list in tests:
    word_id_list = []
    n = 0
    for i, single_pinyin in enumerate(pinyin_list):
        single_pinyin_words = pinyin2word[single_pinyin]
        if n < len(single_pinyin_words):
            n = len(single_pinyin_words)
        word_id_list.append([word2id[single_word] for single_word in single_pinyin_words])

    words = viterbi_use_kenlm_model(word_id_list, pinyin_list, n)
    print(words)

中国人民
明天高考
后天就要出成记了
希望你后天高考顺利
我明天要吃平果
你的报价态高了
秦老勇感
心知度明
金天转了好多前
你是否友好多问好


In [9]:
tests = [
    ['zhong1', 'guo2', 'ren2', 'min2'],
    ['ming2', 'tian1', 'gao1', 'kao3'],
    ['hou4', 'tian1', 'jiu4', 'yao4', 'chu1', 'cheng2', 'ji4', 'le5'],
    ['xi1', 'wang4', 'ni3', 'hou4', 'tian1', 'gao1', 'kao3', 'shun4', 'li4'],
    ['wo3', 'ming2', 'tian1', 'yao4', 'chi1', 'ping2', 'guo3'],
    ['ni3', 'de5', 'bao4', 'jia4', 'tai4', 'gao1', 'le5'],
    ['qin2', 'lao2', 'yong3', 'gan3'],
    ['xin1', 'zhi1', 'du4', 'ming2'],
    ['jin1', 'tian1', 'zhuan4', 'le5', "hao3", 'duo1', 'qian2'],
    ['ni3', 'shi4', 'fou3', 'you3', "hao3", 'duo1', 'wen4', 'hao4']
]

for pinyin_list in tests:
    word_id_list = []
    n = 0
    for i, single_pinyin in enumerate(pinyin_list):
        single_pinyin_words = pinyin2word[single_pinyin]
        if n < len(single_pinyin_words):
            n = len(single_pinyin_words)
        word_id_list.append([word2id[single_word] for single_word in single_pinyin_words])

    words = viterbi_use_kenlm_model_v1(word_id_list, pinyin_list, n)
    print(words)

中国人民
明天高考
后天就要出成绩了
希望你后天高考顺利
我明天要吃苹果
你的报价太高了
勤劳勇感
心知肚明
金天转了好多钱
你是否有好多问号


## pycorrector

使用了KenLm制作的大语料库的n-gram模型后，结果比之前的还是要好了不少，但是任然有一些不尽如人意的地方，比如“金天转了好多钱”就是一个明显的谐音字错误。

而开源的pycorrector是一个中文文本纠错工具。音似、形似错字（或变体字）纠正，可用于中文拼音、笔画输入法的错误纠正。pycorrector依据语言模型检测错别字位置，通过拼音音似特征、笔画五笔编辑距离特征及语言模型困惑度特征纠正错别字。

我们常见错误类型包括：

* 谐音字词，如 配副眼睛-配副眼镜
* 混淆音字词，如 流浪织女-牛郎织女
* 字词顺序颠倒，如 伍迪艾伦-艾伦伍迪
* 字词补全，如 爱有天意-假如爱有天意
* 形似字错误，如 高梁-高粱
* 中文拼音全拼，如 xingfu-幸福
* 中文拼音缩写，如 sz-深圳
* 语法错误，如 想象难以-难以想象

当然，针对不同业务场景，这些问题并不一定全部存在，比如输入法中需要处理前四种，搜索引擎需要处理所有类型，语音识别后文本纠错只需要处理前两种， 其中'形似字错误'主要针对五笔或者笔画手写输入等。

### pycorrector纠错解决方案

**方案一：规则的解决思路**

1. 中文纠错分为两步走，第一步是错误检测，第二步是错误纠正；
2. 错误检测部分先通过结巴中文分词器切词，由于句子中含有错别字，所以切词结果往往会有切分错误的情况，这样从字粒度和词粒度两方面检测错误， 整合这两种粒度的疑似错误结果，形成疑似错误位置候选集；
3. 错误纠正部分，是遍历所有的疑似错误位置，并使用音似、形似词典替换错误位置的词，然后通过语言模型计算句子困惑度，对所有候选集结果比较并排序，得到最优纠正词。

**方案二：深度模型的解决思路**
1. 端到端的深度模型可以避免人工提取特征，减少人工工作量，RNN序列模型对文本任务拟合能力强，rnn_attention在英文文本纠错比赛中取得第一名成绩，证明应用效果不错；
2. CRF会计算全局最优输出节点的条件概率，对句子中特定错误类型的检测，会根据整句话判定该错误，阿里参赛2016中文语法纠错任务并取得第一名，证明应用效果不错；
3. seq2seq模型是使用encoder-decoder结构解决序列转换问题，目前在序列转换任务中（如机器翻译、对话生成、文本摘要、图像描述）使用最广泛、效果最好的模型之一。

In [15]:
import os
from pycorrector import Corrector

words = '金天转了好多钱'
model = Corrector(language_model_path=os.path.join(os.path.dirname(__file__), 'data', 'zh_giga.no_cna_cmn.prune01244.klm'))
corrected_sent, detail = model.correct(words)
print(corrected_sent)

ModuleNotFoundError: No module named 'pycorrector'